Please note that if the plots are not visible, it is because Plotly plots may not display in a new notebook. Please rerun the entire code, which should take about 5-6 seconds.

---
# Markowitz Portfolio Optimization
---

## Importing necessary modules and libraries

In [40]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.express as px
import cvxpy as cp
import numpy as np
import plotly.graph_objects as go
import ast

---
## Gathering the closing price data for 10 selected companies (risky assets) over the last 3 months

In [42]:
# Below is the code for gathering the closing price data for the 10 selected companies (risky assets) over the last 3 months
# The data was downloaded (via Yahoo Finance), cleaned and stored as "assets.csv"
'''
def get_historical_data(tickers, start_date, end_date):
    data = {}  # to store all the data

    # Looping through all the companies to get/download the desired data
    for ticker in tickers:
        # Fetching the company name in proper short format
        company_name = yf.Ticker(ticker).info['longName'].split()[0].strip(',')
        stock_data = yf.download(ticker, start=start_date, end=end_date)  # Downloading the data
        data[company_name] = stock_data['Close']

    # Converting to a pandas dataframe
    data = pd.DataFrame(data)

    return data

# Selected Companties: Apple, Google, Microsoft, Amazon, Tesla, Meta, Nvidia, Paypal, Netflix, Visa
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'META', 'NVDA', 'PYPL', 'NFLX', 'V']

time_period = 90  # 3 months = 90 days
end_date = datetime.today().strftime('%Y-%m-%d')  # Current timestamp
start_date = (datetime.today() - timedelta(days=time_period)).strftime('%Y-%m-%d')  # Start date will be 3 months before the current time stamp

historical_data = get_historical_data(tickers, start_date, end_date)

# Renaming Amazon.com to Amazon and Alphabet to Google (to ensure consistency in naming)
historical_data.rename(columns={
    'Amazon.com': 'Amazon',
    'Alphabet': 'Google'
}, inplace=True)

historical_data.to_csv('assets.csv')
'''
pass

In [43]:
pd.read_csv('assets.csv')

,Date,Apple,Google,Microsoft,Amazon,Tesla,Meta,NVIDIA,PayPal,Netflix,Visa
0,2023-11-06,179.229996,130.250000,356.529999,139.740005,219.270004,315.799988,457.510010,54.619999,434.739990,243.490005
1,2023-11-07,181.820007,130.970001,360.529999,142.710007,222.179993,318.820007,459.549988,54.630001,434.609985,244.770004
2,2023-11-08,182.889999,131.839996,363.200012,142.080002,222.110001,319.779999,465.739990,55.080002,436.649994,243.910004
3,2023-11-09,182.410004,130.240005,360.690002,140.600006,209.979996,320.549988,469.500000,54.279999,435.149994,241.639999
4,2023-11-10,186.399994,132.589996,369.670013,143.559998,214.649994,328.769989,483.350006,54.770000,447.239990,245.250000
...,...,...,...,...,...,...,...,...,...,...,...
56,2024-01-29,191.729996,153.509995,409.720001,161.259995,190.929993,401.019989,624.650024,63.759998,575.789978,273.660004
57,2024-01-30,188.039993,151.460007,408.589996,159.000000,191.589996,400.059998,627.739990,63.680000,562.849976,277.149994
58,2024-01-31,184.399994,140.100006,397.579987,155.199997,187.289993,390.140015,615.270020,61.349998,564.109985,273.260010
59,2024-02-01,186.860001,141.160004,403.779999,159.279999,188.860001,394.779999,630.270020,62.020000,567.510010,277.049988


In [44]:
# Loading the assets dataframe
historical_data = pd.read_csv('assets.csv')
historical_data['Date'] = pd.to_datetime(historical_data['Date'])
historical_data.set_index('Date', inplace=True)

In [45]:
historical_data.head()

,Apple,Google,Microsoft,Amazon,Tesla,Meta,NVIDIA,PayPal,Netflix,Visa
Date,,,,,,,,,,
2023-11-06,179.229996,130.250000,356.529999,139.740005,219.270004,315.799988,457.510010,54.619999,434.739990,243.490005
2023-11-07,181.820007,130.970001,360.529999,142.710007,222.179993,318.820007,459.549988,54.630001,434.609985,244.770004
2023-11-08,182.889999,131.839996,363.200012,142.080002,222.110001,319.779999,465.739990,55.080002,436.649994,243.910004
2023-11-09,182.410004,130.240005,360.690002,140.600006,209.979996,320.549988,469.500000,54.279999,435.149994,241.639999
2023-11-10,186.399994,132.589996,369.670013,143.559998,214.649994,328.769989,483.350006,54.770000,447.239990,245.250000


---
## Calculating the returns and risk measures for each of the 10 assests

In [46]:
def calculate_returns(historical_data):
    returns = historical_data.pct_change().dropna()  # Percentage change
    for column_name in returns.columns:
        returns.rename(columns={
            column_name: column_name + '_Return'
        }, inplace=True)
    return returns

In [47]:
def calculate_risks(historical_data):
    risks = historical_data.pct_change().dropna().std()
    return risks

In [48]:
assets_risk = calculate_risks(historical_data)

### Displaying the risk of each assets

In [49]:
assets_risk

Apple        0.011458
Google       0.016344
Microsoft    0.010810
Amazon       0.016321
Tesla        0.027877
Meta         0.029283
NVIDIA       0.019487
PayPal       0.023020
Netflix      0.019923
Visa         0.007228
dtype: float64

In [50]:
return_data = calculate_returns(historical_data)

In [51]:
return_data.tail(10)

,Apple_Return,Google_Return,Microsoft_Return,Amazon_Return,Tesla_Return,Meta_Return,NVIDIA_Return,PayPal_Return,Netflix_Return,Visa_Return
Date,,,,,,,,,,
2024-01-22,0.012163,-0.002664,-0.005418,-0.003605,-0.015976,-0.004355,0.002740,-0.031601,0.005715,0.001107
2024-01-23,0.006653,0.007192,0.006028,0.008011,0.001628,0.008958,0.003671,0.016316,0.013341,0.000221
2024-01-24,-0.003484,0.011289,0.009175,0.005448,-0.006264,0.014278,0.024869,-0.027169,0.107032,0.001438
2024-01-25,-0.001697,0.021318,0.005738,0.005610,-0.121253,0.006348,0.004156,-0.036655,0.031439,0.003534
2024-01-26,-0.009013,0.002107,-0.002322,0.008685,0.003395,0.002442,-0.009510,0.017625,0.014982,-0.017131
2024-01-29,-0.003586,0.008673,0.014334,0.013449,0.041910,0.017456,0.023496,0.032049,0.009414,0.021348
2024-01-30,-0.019246,-0.013354,-0.002758,-0.014015,0.003457,-0.002394,0.004947,-0.001255,-0.022473,0.012753
2024-01-31,-0.019358,-0.075003,-0.026946,-0.023899,-0.022444,-0.024796,-0.019865,-0.036589,0.002239,-0.014036
2024-02-01,0.013341,0.007566,0.015594,0.026289,0.008383,0.011893,0.024380,0.010921,0.006027,0.013869


### Visualising the returns

In [52]:
viz_data = return_data.reset_index()
viz_data['Date'] = viz_data['Date'].dt.date

In [53]:
viz_data.set_index('Date', inplace=True)

In [54]:
px.imshow(viz_data, height=800, width=1200)

In [55]:
viz_data.reset_index(inplace=True)

In [56]:
melted_data = viz_data.melt(id_vars='Date', var_name='Company', value_name='Return')

# Create a line plot
fig = px.line(melted_data, x='Date', y='Return', color='Company', title='Company Returns Over Time')
fig.update_layout(yaxis=dict(range=[-0.11, 0.11]), title_text="Company Returns Over Time")


# Show the plot
fig.show()

---
## Markowitz's Mean-Variance Optimization

### Using the covariance matrix as the risk model

In [57]:
Σ = return_data.cov()  # Covariance Matrix (Risk Model)
μ = return_data.mean()  # Expected Return

In [58]:
μ

Apple_Return        0.000669
Google_Return       0.001618
Microsoft_Return    0.002439
Amazon_Return       0.003578
Tesla_Return       -0.002178
Meta_Return         0.007209
NVIDIA_Return       0.006352
PayPal_Return       0.002487
Netflix_Return      0.004556
Visa_Return         0.002188
dtype: float64

In [59]:
Σ

,Apple_Return,Google_Return,Microsoft_Return,Amazon_Return,Tesla_Return,Meta_Return,NVIDIA_Return,PayPal_Return,Netflix_Return,Visa_Return
Apple_Return,0.000131,0.000092,0.000071,0.000083,0.000053,0.000068,0.000110,0.000094,0.000071,0.000034
Google_Return,0.000092,0.000267,0.000114,0.000134,0.000032,0.000160,0.000150,0.000056,0.000100,0.000040
Microsoft_Return,0.000071,0.000114,0.000117,0.000112,0.000015,0.000155,0.000132,0.000053,0.000078,0.000039
Amazon_Return,0.000083,0.000134,0.000112,0.000266,0.000054,0.000372,0.000200,0.000094,0.000072,0.000037
Tesla_Return,0.000053,0.000032,0.000015,0.000054,0.000777,0.000062,0.000114,0.000274,-0.000033,0.000040
Meta_Return,0.000068,0.000160,0.000155,0.000372,0.000062,0.000858,0.000302,0.000131,0.000070,0.000035
NVIDIA_Return,0.000110,0.000150,0.000132,0.000200,0.000114,0.000302,0.000380,0.000074,0.000112,0.000059
PayPal_Return,0.000094,0.000056,0.000053,0.000094,0.000274,0.000131,0.000074,0.000530,0.000006,0.000046
Netflix_Return,0.000071,0.000100,0.000078,0.000072,-0.000033,0.000070,0.000112,0.000006,0.000397,0.000032
Visa_Return,0.000034,0.000040,0.000039,0.000037,0.000040,0.000035,0.000059,0.000046,0.000032,0.000052


### Expected Return: $E(R) = \Sigma(r_i \times p_i)$

In [60]:
class MarkowitzOptimizer(object):
    def __init__(self, returns):
        self.return_data = return_data  # The returns dataframe
        self.μ = returns.mean()  # Mean return for each company
        self.Σ = returns.cov()  # Covariance Matrix for each company
        self.n_assets = self.return_data.shape[1]  # Number of assests we have

    def get_optimal_results(self):
        '''
        Returns the optimal weights, expected return and risk corresponding to the return series
        '''
        weights = cp.Variable(self.n_assets)  # The variables
        portfolio_risk = cp.quad_form(weights, self.Σ)  # Associated risks

        # The goal is to minimize the following portfolio risk
        objective = cp.Minimize(portfolio_risk)
        # Which are subjected to the following constraints
        constraints = [cp.sum(weights) == 1, weights >= 1e-16, weights <= 1]

        # Solving the optimization problem
        problem = cp.Problem(objective, constraints)
        problem.solve()

        optimal_weights = weights.value  # These are the optimal weights

        # Get the corresponding portfolio optimal return and risk
        optimal_portfolio_return = self.μ.dot(optimal_weights)
        optimal_portfolio_risk = np.sqrt(portfolio_risk.value)
        self.portfolio_risk = portfolio_risk

        return optimal_weights, optimal_portfolio_return, optimal_portfolio_risk, weights


    def plot_efficient_frontier(self, n_points=500):
        '''
        This method will plot the efficient frontier with number of scatter points = n_points
        '''
        returns = np.linspace(self.μ.min(), self.μ.max(), n_points)
        self.returns = np.array(returns)

        optimal_weights, optimal_portfolio_return, optimal_portfolio_risk, weights = self.get_optimal_results()
        risks = []
        all_weights = []
        for return_ in returns:

            # Defining the constraints for a given target return
            constraints = [
                cp.sum(weights) == 1,
                weights >= 1e-16,  # Nearly 0, set to very small value due to precision error of library
                weights <= 1,
                cp.sum(cp.multiply(self.μ, weights)) == return_
            ]

            # Solving the optimization problem
            problem = cp.Problem(cp.Minimize(self.portfolio_risk), constraints)
            problem.solve()

            risk = np.sqrt(self.portfolio_risk.value)
            risks.append(risk)
            all_weights.append(str([round(wt, 3) for wt in weights.value]))

        efficient_frontier_data = pd.DataFrame({'Risk': risks, 'Return': returns, 'Weights': all_weights})
        self.all_weights = np.array(all_weights)

        # Plotting the efficient Frontier
        fig = px.scatter(
            efficient_frontier_data,
            x='Risk',
            y='Return',
            title='Markowitz Efficient Frontier',
            labels={'Risk': 'Portfolio Risk', 'Return': 'Portfolio Return'},
            hover_data={'Risk': True, 'Return': True, 'Weights': True},
            height=500,
            width=1000
        )
        fig.update_layout(
            hoverlabel=dict(
                font_size=8
            )
        )

        optimal_point = go.Scatter(
            x=[optimal_portfolio_risk],
            y=[optimal_portfolio_return],
            mode='markers',
            marker=dict(color='purple', size=12, symbol='star'),
            name='Global Minimum Risk',
            hoverinfo='text',
            text=['Optimal Risk: {:.4f}'.format(optimal_portfolio_risk) + ' & Optimal Return: {:.4f}'.format(optimal_portfolio_return)]
        )

        fig.update_traces(marker=dict(color='cornflowerblue'))
        fig.add_trace(optimal_point)

        self.risks = np.array(risks)
        return fig

    def plot_upper_half(self, n_points=100):
        '''
        This method will plot the efficient frontier(upper half only) with number of scatter points = n_points
        '''
        optimal_weights, optimal_portfolio_return, optimal_portfolio_risk, weights = self.get_optimal_results()
        returns = np.linspace(optimal_portfolio_return, self.μ.max(), n_points)
        self.returns_upper_half = np.array(returns)


        risks = []
        all_weights = []
        for return_ in returns:

            # Defining the constraints for a given target return
            constraints = [
                cp.sum(weights) == 1,
                weights >= 1e-16,
                weights <= 1,
                cp.sum(cp.multiply(self.μ, weights)) == return_
            ]

            # Solving the optimization problem
            problem = cp.Problem(cp.Minimize(self.portfolio_risk), constraints)
            problem.solve()

            risk = np.sqrt(self.portfolio_risk.value)
            risks.append(risk)
            all_weights.append(str([round(wt, 3) for wt in weights.value]))

        efficient_frontier_data = pd.DataFrame({'Risk': risks, 'Return': returns, 'Weights': all_weights})
        # self.all_weights_upper_half = np.array(all_weights)
        self.all_weights_upper_half = np.array(all_weights)

        # Plotting the efficient Frontier
        fig = px.scatter(
            efficient_frontier_data,
            x='Risk',
            y='Return',
            title='Markowitz Efficient Frontier',
            labels={'Risk': 'Portfolio Risk', 'Return': 'Portfolio Return'},
            hover_data={'Risk': True, 'Return': True, 'Weights': True},
            height=500,
            width=1000
        )
        fig.update_layout(
            hoverlabel=dict(
                font_size=8
            )
        )

        optimal_point = go.Scatter(
            x=[optimal_portfolio_risk],
            y=[optimal_portfolio_return],
            mode='markers',
            marker=dict(color='purple', size=12, symbol='star'),
            name='Global Minimum Risk',
            hoverinfo='text',
            text=['Optimal Risk: {:.4f}'.format(optimal_portfolio_risk) + ' & Optimal Return: {:.4f}'.format(optimal_portfolio_return)]
        )

        fig.update_traces(marker=dict(color='cornflowerblue'))
        fig.add_trace(optimal_point)

        self.risks_upper_half = np.array(risks)
        return fig

    def plot_efficient_frontier_with_selected_points(self, n_points=500, risk_tolerance1=None, risk_tolerance2=None):
        fig = self.plot_upper_half()
        if risk_tolerance1 is not None:
            # Find the index of the closest value to the specified risk tolerance level 1
            index1 = np.argmin(np.abs(self.risks_upper_half - risk_tolerance1))
            risk1 = self.risks_upper_half[index1]
            return1 = self.returns_upper_half[index1]
            weights1 = self.all_weights_upper_half[index1]

            # Highlight the first risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk1],
                y=[return1],
                mode='markers',
                marker=dict(color='green', size=12, symbol='star'),
                name=f'Risk Tolerance 1 ({risk_tolerance1})',
                hoverinfo='text',
                text=[f'Risk: {risk1:.4f} & Return: {return1:.4f}', f'Weights: {weights1}']
            ))

        if risk_tolerance2 is not None:
            # Find the index of the closest value to the specified risk tolerance level 2
            index2 = np.argmin(np.abs(self.risks_upper_half - risk_tolerance2))
            risk2 = self.risks_upper_half[index2]
            return2 = self.returns_upper_half[index2]
            weights2 = self.all_weights_upper_half[index2]

            # Highlight the second risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk2],
                y=[return2],
                mode='markers',
                marker=dict(color='black', size=12, symbol='star'),
                name=f'Risk Tolerance 2 ({risk_tolerance2})',
                hoverinfo='text',
                text=[f'Risk: {risk2:.4f} & Return: {return2:.4f}', f'Weights: {weights2}']
            ))

        return fig, weights1, weights2

    def plot_points_wrt_expected_return(self, n_points=500, expected_return1=None, expected_return2=None):
        fig = self.plot_upper_half()
        if expected_return1 is not None:
            # Find the index of the closest value to the specified risk tolerance level 1
            index1 = np.argmin(np.abs(self.returns_upper_half - expected_return1))
            risk1 = self.risks_upper_half[index1]
            return1 = self.returns_upper_half[index1]
            weights1 = self.all_weights_upper_half[index1]

            # Highlight the first risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk1],
                y=[return1],
                mode='markers',
                marker=dict(color='green', size=12, symbol='star'),
                name=f'Return 1 ({expected_return1})',
                hoverinfo='text',
                text=[f'Risk: {risk1:.4f} & Return: {return1:.4f}', f'Weights: {weights1}']
            ))

        if expected_return2 is not None:
            # Find the index of the closest value to the specified risk tolerance level 2
            index2 = np.argmin(np.abs(self.returns_upper_half - expected_return2))
            risk2 = self.risks_upper_half[index2]
            return2 = self.returns_upper_half[index2]
            weights2 = self.all_weights_upper_half[index2]

            # Highlight the second risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk2],
                y=[return2],
                mode='markers',
                marker=dict(color='black', size=12, symbol='star'),
                name=f'Return 2 ({expected_return2})',
                hoverinfo='text',
                text=[f'Risk: {risk2:.4f} & Return: {return2:.4f}', f'Weights: {weights2}']
            ))

        return fig, weights1, weights2

In [61]:
opt = MarkowitzOptimizer(return_data)

In [62]:
fig = opt.plot_efficient_frontier(n_points=100)
fig.show()

### Plotting only the upper half of the curve

In [63]:
fig = opt.plot_upper_half()
fig.show()

### Selecting two points on the efficient frontier and calculating the corresponding weights

In [64]:
fig, weights1, weights2 = opt.plot_efficient_frontier_with_selected_points(risk_tolerance1=0.013, risk_tolerance2=0.018)
fig.show()

### Displaying the weights of the two choosen points

In [65]:
weights1 = pd.Series(index=return_data.columns, data=ast.literal_eval(weights1))
weights2 = pd.Series(index=return_data.columns, data=ast.literal_eval(weights2))

In [66]:
print(f'Weights for point 1: \n{weights1}\n\n')
print(f'Weights for point 2: \n{weights2}')

Weights for point 1: 
Apple_Return        0.000
Google_Return       0.000
Microsoft_Return    0.000
Amazon_Return       0.000
Tesla_Return        0.000
Meta_Return         0.147
NVIDIA_Return       0.376
PayPal_Return       0.001
Netflix_Return      0.225
Visa_Return         0.252
dtype: float64


Weights for point 2: 
Apple_Return        0.000
Google_Return       0.000
Microsoft_Return    0.000
Amazon_Return       0.000
Tesla_Return        0.000
Meta_Return         0.253
NVIDIA_Return       0.610
PayPal_Return       0.000
Netflix_Return      0.137
Visa_Return         0.000
dtype: float64


---
## **Additional:** Calculating the global minimum risk for a desired return value

In [67]:
fig, weights1, weights2 = opt.plot_points_wrt_expected_return(expected_return1=0.0032, expected_return2=0.0060)
fig.show()

In [68]:
weights1 = pd.Series(index=return_data.columns, data=ast.literal_eval(weights1))
weights2 = pd.Series(index=return_data.columns, data=ast.literal_eval(weights2))

In [69]:
print(f'Weights for point 1:\n{weights1}\n\n')
print(f'Weights for point 2:\n{weights2}')

Weights for point 1:
Apple_Return        0.000
Google_Return       0.000
Microsoft_Return    0.000
Amazon_Return       0.000
Tesla_Return        0.000
Meta_Return         0.074
NVIDIA_Return       0.086
PayPal_Return       0.007
Netflix_Return      0.117
Visa_Return         0.717
dtype: float64


Weights for point 2:
Apple_Return        0.000
Google_Return       0.000
Microsoft_Return    0.000
Amazon_Return       0.000
Tesla_Return        0.000
Meta_Return         0.187
NVIDIA_Return       0.534
PayPal_Return       0.000
Netflix_Return      0.279
Visa_Return         0.000
dtype: float64


---

---
## **Additional:** Allowing short-selling

In [70]:
class MarkowitzOptimizerShortSelling(object):
    def __init__(self, returns):
        self.return_data = return_data  # The returns dataframe
        self.μ = returns.mean()  # Mean return for each company
        self.Σ = returns.cov()  # Covariance Matrix for each company
        self.n_assets = self.return_data.shape[1]  # Number of assests we have

    def get_optimal_results(self):
        '''
        Returns the optimal weights, expected return and risk corresponding to the return series
        '''
        weights = cp.Variable(self.n_assets)  # The variables
        portfolio_risk = cp.quad_form(weights, self.Σ)  # Associated risks

        # The goal is to minimize the following portfolio risk
        objective = cp.Minimize(portfolio_risk)
        # Which are subjected to the following constraints
        constraints = [cp.sum(weights) == 1]  # No constraint on weight being 0 to 1

        # Solving the optimization problem
        problem = cp.Problem(objective, constraints)
        problem.solve()

        optimal_weights = weights.value  # These are the optimal weights

        # Get the corresponding portfolio optimal return and risk
        optimal_portfolio_return = self.μ.dot(optimal_weights)
        optimal_portfolio_risk = np.sqrt(portfolio_risk.value)
        self.portfolio_risk = portfolio_risk

        return optimal_weights, optimal_portfolio_return, optimal_portfolio_risk, weights


    def plot_efficient_frontier(self, n_points=500):
        '''
        This method will plot the efficient frontier with number of scatter points = n_points
        '''
        returns = np.linspace(self.μ.min(), self.μ.max(), n_points)
        self.returns = np.array(returns)

        optimal_weights, optimal_portfolio_return, optimal_portfolio_risk, weights = self.get_optimal_results()
        risks = []
        all_weights = []
        for return_ in returns:

            # Defining the constraints for a given target return
            constraints = [
                cp.sum(weights) == 1,
                cp.sum(cp.multiply(self.μ, weights)) == return_
            ]

            # Solving the optimization problem
            problem = cp.Problem(cp.Minimize(self.portfolio_risk), constraints)
            problem.solve()

            risk = np.sqrt(self.portfolio_risk.value)
            risks.append(risk)
            all_weights.append(str([round(wt, 3) for wt in weights.value]))

        efficient_frontier_data = pd.DataFrame({'Risk': risks, 'Return': returns, 'Weights': all_weights})
        self.all_weights = np.array(all_weights)

        # Plotting the efficient Frontier
        fig = px.scatter(
            efficient_frontier_data,
            x='Risk',
            y='Return',
            title='Markowitz Efficient Frontier',
            labels={'Risk': 'Portfolio Risk', 'Return': 'Portfolio Return'},
            hover_data={'Risk': True, 'Return': True, 'Weights': True},
            height=500,
            width=1000
        )
        fig.update_layout(
            hoverlabel=dict(
                font_size=8
            )
        )

        optimal_point = go.Scatter(
            x=[optimal_portfolio_risk],
            y=[optimal_portfolio_return],
            mode='markers',
            marker=dict(color='purple', size=12, symbol='star'),
            name='Global Minimum Risk',
            hoverinfo='text',
            text=['Optimal Risk: {:.4f}'.format(optimal_portfolio_risk) + ' & Optimal Return: {:.4f}'.format(optimal_portfolio_return)]
        )

        fig.update_traces(marker=dict(color='cornflowerblue'))
        fig.add_trace(optimal_point)

        self.risks = np.array(risks)
        return fig

    def plot_upper_half(self, n_points=100):
        '''
        This method will plot the efficient frontier(upper half only) with number of scatter points = n_points
        '''
        optimal_weights, optimal_portfolio_return, optimal_portfolio_risk, weights = self.get_optimal_results()
        returns = np.linspace(optimal_portfolio_return, self.μ.max(), n_points)
        self.returns_upper_half = np.array(returns)


        risks = []
        all_weights = []
        for return_ in returns:

            # Defining the constraints for a given target return
            constraints = [
                cp.sum(weights) == 1,
                cp.sum(cp.multiply(self.μ, weights)) == return_
            ]

            # Solving the optimization problem
            problem = cp.Problem(cp.Minimize(self.portfolio_risk), constraints)
            problem.solve()

            risk = np.sqrt(self.portfolio_risk.value)
            risks.append(risk)
            all_weights.append(str([round(wt, 3) for wt in weights.value]))

        efficient_frontier_data = pd.DataFrame({'Risk': risks, 'Return': returns, 'Weights': all_weights})
        # self.all_weights_upper_half = np.array(all_weights)
        self.all_weights_upper_half = np.array(all_weights)

        # Plotting the efficient Frontier
        fig = px.scatter(
            efficient_frontier_data,
            x='Risk',
            y='Return',
            title='Markowitz Efficient Frontier',
            labels={'Risk': 'Portfolio Risk', 'Return': 'Portfolio Return'},
            hover_data={'Risk': True, 'Return': True, 'Weights': True},
            height=500,
            width=1000
        )
        fig.update_layout(
            hoverlabel=dict(
                font_size=8
            )
        )

        optimal_point = go.Scatter(
            x=[optimal_portfolio_risk],
            y=[optimal_portfolio_return],
            mode='markers',
            marker=dict(color='purple', size=12, symbol='star'),
            name='Global Minimum Risk',
            hoverinfo='text',
            text=['Optimal Risk: {:.4f}'.format(optimal_portfolio_risk) + ' & Optimal Return: {:.4f}'.format(optimal_portfolio_return)]
        )

        fig.update_traces(marker=dict(color='cornflowerblue'))
        fig.add_trace(optimal_point)

        self.risks_upper_half = np.array(risks)
        return fig

    def plot_efficient_frontier_with_selected_points(self, n_points=500, risk_tolerance1=None, risk_tolerance2=None):
        fig = self.plot_upper_half()
        if risk_tolerance1 is not None:
            # Find the index of the closest value to the specified risk tolerance level 1
            index1 = np.argmin(np.abs(self.risks_upper_half - risk_tolerance1))
            risk1 = self.risks_upper_half[index1]
            return1 = self.returns_upper_half[index1]
            weights1 = self.all_weights_upper_half[index1]

            # Highlight the first risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk1],
                y=[return1],
                mode='markers',
                marker=dict(color='green', size=12, symbol='star'),
                name=f'Risk Tolerance 1 ({risk_tolerance1})',
                hoverinfo='text',
                text=[f'Risk: {risk1:.4f} & Return: {return1:.4f}', f'Weights: {weights1}']
            ))

        if risk_tolerance2 is not None:
            # Find the index of the closest value to the specified risk tolerance level 2
            index2 = np.argmin(np.abs(self.risks_upper_half - risk_tolerance2))
            risk2 = self.risks_upper_half[index2]
            return2 = self.returns_upper_half[index2]
            weights2 = self.all_weights_upper_half[index2]

            # Highlight the second risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk2],
                y=[return2],
                mode='markers',
                marker=dict(color='black', size=12, symbol='star'),
                name=f'Risk Tolerance 2 ({risk_tolerance2})',
                hoverinfo='text',
                text=[f'Risk: {risk2:.4f} & Return: {return2:.4f}', f'Weights: {weights2}']
            ))

        return fig, weights1, weights2

    def plot_points_wrt_expected_return(self, n_points=500, expected_return1=None, expected_return2=None):
        fig = self.plot_upper_half()
        if expected_return1 is not None:
            # Find the index of the closest value to the specified risk tolerance level 1
            index1 = np.argmin(np.abs(self.returns_upper_half - expected_return1))
            risk1 = self.risks_upper_half[index1]
            return1 = self.returns_upper_half[index1]
            weights1 = self.all_weights_upper_half[index1]

            # Highlight the first risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk1],
                y=[return1],
                mode='markers',
                marker=dict(color='green', size=12, symbol='star'),
                name=f'Return 1 ({expected_return1})',
                hoverinfo='text',
                text=[f'Risk: {risk1:.4f} & Return: {return1:.4f}', f'Weights: {weights1}']
            ))

        if expected_return2 is not None:
            # Find the index of the closest value to the specified risk tolerance level 2
            index2 = np.argmin(np.abs(self.returns_upper_half - expected_return2))
            risk2 = self.risks_upper_half[index2]
            return2 = self.returns_upper_half[index2]
            weights2 = self.all_weights_upper_half[index2]

            # Highlight the second risk tolerance level point with a different color
            fig.add_trace(go.Scatter(
                x=[risk2],
                y=[return2],
                mode='markers',
                marker=dict(color='black', size=12, symbol='star'),
                name=f'Return 2 ({expected_return2})',
                hoverinfo='text',
                text=[f'Risk: {risk2:.4f} & Return: {return2:.4f}', f'Weights: {weights2}']
            ))

        return fig, weights1, weights2

In [71]:
opt = MarkowitzOptimizerShortSelling(return_data)

In [72]:
fig = opt.plot_efficient_frontier(n_points=100)
fig.show()

In [73]:
fig, weights1, weights2 = opt.plot_efficient_frontier_with_selected_points(risk_tolerance1=0.007, risk_tolerance2=0.009)
fig.show()

In [74]:
weights1 = pd.Series(index=return_data.columns, data=ast.literal_eval(weights1))
weights2 = pd.Series(index=return_data.columns, data=ast.literal_eval(weights2))

In [75]:
weights1

Apple_Return        0.049
Google_Return      -0.043
Microsoft_Return    0.095
Amazon_Return       0.015
Tesla_Return       -0.005
Meta_Return         0.037
NVIDIA_Return      -0.039
PayPal_Return       0.003
Netflix_Return      0.066
Visa_Return         0.821
dtype: float64

In [76]:
weights2

Apple_Return       -0.173
Google_Return      -0.083
Microsoft_Return    0.022
Amazon_Return      -0.026
Tesla_Return       -0.086
Meta_Return         0.071
NVIDIA_Return       0.153
PayPal_Return       0.075
Netflix_Return      0.131
Visa_Return         0.916
dtype: float64

---